In [1]:
import requests
import json
import facebook
import re
from datetime import datetime
import concurrent.futures
from pymongo import MongoClient

In [46]:
# fan_page = "DoctorKoWJ"

token = input("your token")
person  = "DoctorKoWJ"
group_id = requests.get('https://graph.facebook.com/?id={}&access_token={}'.format(person, token)).json()["id"]
graph = facebook.GraphAPI(access_token = token)

client = MongoClient("mongodb//localhost:27017")
db = client["politics"]

In [35]:
def get_comments(post_id):
    comments = []
    obj_page = graph.request(post_id, {'fields':'comments{created_time,id,message,likes.limit(0).summary(True)}'})["comments"]
    
    while True:
        for comment in obj_page["data"]:
            comment["created_time"] = datetime.strptime(comment["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
            comment["likes"] = comment["likes"]["summary"]["total_count"]
            comment["post_id"] = post_id
            comments.append(comment)
            print("finish {}".format(comment["id"]))


        if "next" in obj_page["paging"]:
            next_page = obj_page["paging"]["next"]
            obj_page = requests.get(next_page).json()
        else:
            break
        
    return comments

In [ ]:
def get_posts():
    kp_posts = []
    obj_page = graph.request(group_id,{'fields':'posts.since(2017-11-20){created_time, message,id,likes.limit(0).summary(True),shares,link}'})["posts"]

    while True:
        for post in obj_page["data"]:
            post["created_time"] = datetime.strptime(post["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
            post["likes"] = post["likes"]["summary"]["total_count"]
            post["shares"] = post["shares"]["count"]
            kp_posts.append(post)
            
            print("finish {}".format(post["id"]))

        if "next" in obj_page["paging"]:
            next_page = obj_page["paging"]["next"]
            obj_page = requests.get(next_page).json()
        else:
            break
    
    return kp_posts

In [ ]:
def main():
    kp_posts = get_posts()
    posts_collections = db["posts"]
    posts_collections.insert_many(kp_posts)
    post_ids = [post["id"] for post in kp_posts]
    
    kp_comments_collections = db["kp_comments"]
    for post_id in post_ids:
        comments = get_comments(post_id)
        kp_comments_collections.insert_many(comments)

In [ ]:
if __name__ == "__main__":
    main()

In [49]:
# function (get_posts) test

kp_posts = []
obj_page = graph.request(group_id,{'fields':'posts.since(2017-11-01){created_time, message,id,likes.limit(0).summary(True),shares, reactions{type}}'})["posts"]

for _ in range(1):
    for post in obj_page["data"]:
        post["created_time"] = datetime.strptime(post["created_time"], "%Y-%m-%dT%H:%M:%S+0000")
        post["likes"] = post["likes"]["summary"]["total_count"]
        post["shares"] = post["shares"]["count"]
        kp_posts.append(post)
        
    
    if "next" in obj_page["paging"]:
        next_page = obj_page["paging"]["next"]
        obj_page = requests.get(next_page).json()
    else:
        break
        


In [50]:
kp_posts

[{'created_time': datetime.datetime(2017, 11, 24, 10, 37, 11),
  'id': '136845026417486_1176168899151755',
  'likes': 19919,
  'message': '中和的一場大火帶走9條人命，令人遺憾的是，其中一位罹難者還是我們北市的警察同仁。\n\n這場大火，燒出了違建背後的安全問題，台北市也必須嚴肅面對。\n\n3年前，我們曾經針對頂樓有危害公共安全的違建祭出「226專案」，不到3個月時間就讓226處違建拆光。上個月，我們再度針對新增的頂樓加蓋違建公告「208頂樓專案」，要求208處違建屋主在今年11月15日前自行改善，否則就會被強制拆除，多數屋主都願意配合。\n\n只是即便如此，違建火災事件還是層出不窮，因此，我們對於既有的頂樓加蓋也要祭出新規定，只要是未加裝住警器（住宅用火災警報器）的違建，名單將由消防局移送建管處，納入「優先拆除對象」。\n\n排除新的違建較容易，但要改善數萬戶既有的「歷史共業」則是需要時間。對於打擊違建，我們態度不變，但最重要的還是要建立一個守法的氛圍，不要讓違建一再出現，才是根本解決之道。\n\n---\n台北市頂樓加蓋處所安裝住警器 Q&A\nhttps://goo.gl/7iYJDr',
  'shares': 289},
 {'created_time': datetime.datetime(2017, 11, 21, 12, 42, 23),
  'id': '136845026417486_1173911382710840',
  'likes': 10545,
  'message': '以前，政府興建公共住宅，只是當作一個「可以住的地方」，外觀像火柴盒一樣缺乏美感，對社區也缺乏連結與回饋，導致很多居民對公宅有很大的成見，甚至反對。\n\n為了扭轉社會對於公宅的刻板印象，我要求每一座公宅不只外觀要有設計感，還要導入智慧電網、智慧電錶、智慧水錶、智慧瓦斯錶等科技設備，低樓層更要配置完善的托嬰、托幼、托老、民眾活動中心等公共設施，創造更多高質感、高科技、高機能的新時代公宅。\n\n最重要，我們要讓公宅成為社區居民的「好厝邊」，所以我們不只請專人協助美化社區或水溝清淤，更計畫讓入住的青年，可以為社區提出一些想法，使公宅

In [ ]:
# test
posts = graph.get_connections(id = 'me', connection_name='friends')
graph.get_connections(id = 'me', connection_name='friends', limit=100, fields=['message', 'id', 'from', 'type', 'picture', 'link', 'created_time', 'updated_time'])